In [2]:
import numpy as np
from scipy.stats import t
from scipy.special import betainc

In [4]:
group1 = [72, 73, 73, 77, 82, 82, 83, 84, 85]
group2 = [81, 82, 83, 83, 83, 84, 87, 90, 92, 93]
group3 = [77, 78, 79, 88, 89, 90, 91, 95, 95, 98]

data = np.array([group1, group2, group3], dtype=object)

In [5]:
data

array([list([72, 73, 73, 77, 82, 82, 83, 84, 85]),
       list([81, 82, 83, 83, 83, 84, 87, 90, 92, 93]),
       list([77, 78, 79, 88, 89, 90, 91, 95, 95, 98])], dtype=object)

In [19]:
# Functions required for calculations of ANOVA statistics

# sum of squares of treatment / regression
def ssr(data):
    sums = []
    for group in data:
        sums.append(len(group) * (np.mean(group) - np.mean(data))**2)
        
    return np.sum(sums)

# sum of squares error
def sse(data):
    sums = []
    for group in data:
        sums.append(np.sum([(x-np.mean(group))**2 for x in group]))
    
    return np.sum(sums)

def dfr(data):
    return np.shape(data)[0] - 1
    
def dfe(data):
    if np.ndim(data) == 1:
        return np.sum([len(x) for x in data]) - len(data)
    elif np.ndim(data) == 2:
        return np.size(data) - np.shape(data)[0]

def msr(data):
    return ssr(data) / dfr(data)

def mse(data):
    return sse(data) / dfe(data)

def fval(data):
    return msr(data) / mse(data)

def pval(data):
    """
    Finds p-value associated with combination of F and df values.
    
    Uses incomplete betfunction from scipy. Also discussed here:
    https://stackoverflow.com/questions/38113120/calculate-f-distribution-p-values-in-python
    """
    F, df1, df2 = fval(data), dfr(data), dfe(data)
    return 1 - betainc(.5*df1, .5*df2, float(df1)*F/(df1*F+df2))

def lsd(data, groups, alpha=0.025):
    return t.ppf(1-alpha, dfe(data)) * np.sqrt(mse(data) * (1/len(data[groups[0]]) + 1/len(data[groups[1]])))

def lsd_pval(data, groups):
    """
    Returns two-tailed p-value for LSD (least significant difference) posthoc tests.
    
    Args
    data : Numpy array
        Data to be tested, m x n array where first dimension are groups (between) and second dim is subjects (within)
    groups : List or tuple
        Two integers specifying groups to be compared.
        
    Returns
        
    """
    if np.ndim(data) == 1:
        gp1 = data[groups[0]]
        gp2 = data[groups[1]]
    elif np.ndim(data) == 2:
        gp1 = data[groups[0],:]
        gp2 = data[groups[1],:]
    else:
        print("Input data has too many dimensions. Exiting.")
        return
    
    diff = np.abs(np.mean(gp1) - np.mean(gp2))
    
    tval = diff / np.sqrt(mse(data) * (1/len(gp1) + 1/len(gp2)))
    pval = t.sf(np.abs(tval), dfe(data))*2
    
    return pval

# group1 = [72, 73, 73, 77, 82, 82, 83, 84, 85, 89]
# group2 = [81, 82, 83, 83, 83, 84, 87, 90, 92, 93]
# group3 = [77, 78, 79, 88, 89, 90, 91, 95, 95, 98]

# data = np.array([group1, group2, group3])

group1 = [72, 73, 73, 77, 82, 82, 83, 84, 85]
group2 = [81, 82, 83, 83, 83, 84, 87, 90, 92, 93]
group3 = [77, 78, 79, 88, 89, 90, 91, 95, 95, 98]
data = np.array([group1, group2, group3], dtype=object)

lsd_pval(data, [0,2])
# sse(data)

0.0027028893803133056

In [8]:
np.ndim(data)

1